In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

# Load the pre-trained model from PyTorch Hub
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
model.eval()

if torch.cuda.is_available():
    model.to('cuda')

# Define the transformation
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:00<00:00, 174MB/s]


In [3]:
import zipfile
from google.colab import files
import os

# Initialize video capture
video_path = '/content/drive/Shareddrives/TUG dataset/TUG dataset/17/PD/side/PD_side_8.MOV'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error opening video file")
    exit()

output_dir = 'silhouette_frames'
os.makedirs(output_dir, exist_ok=True)
frame_idx = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_image = Image.fromarray(frame_rgb)
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
    with torch.no_grad():
        output = model(input_batch)['out'][0]
    output_predictions = output.argmax(0)
    binary_mask = (output_predictions == 15).byte().cpu().numpy()
    silhouette_image = Image.fromarray((binary_mask * 255).astype(np.uint8))
    silhouette_image.save(os.path.join(output_dir, f'silhouette_frame_{frame_idx}.png'))
    frame_idx += 1

cap.release()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


In [4]:
from google.colab import files as colab_files
# Zip files
zip_filename = 'silhouettes.zip'
zipf = zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED)
for root, dirs, files in os.walk(output_dir):
    for file in files:
        zipf.write(os.path.join(root, file), arcname=file)
zipf.close()

# Download ZIP file
colab_files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>